In [1]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-09-14 01:27:19--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32082132 (31M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  30.60M  22.0MB/s    in 1.4s    

2020-09-14 01:27:20 (22.0 MB/s) - ‘datos_abiertos_covid19.zip’ saved [32082132/32082132]

--2020-09-14 01:27:21--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

Luego, procedemos a cargar los datos.

In [3]:
import statistics

import pandas as pd
import numpy as np

In [4]:
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(nombre_archivo, encoding='latin-1')

In [5]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-09-13,1e6e4c,2,4,25,2,25,25,11,1,2020-04-20,2020-04-17,9999-99-99,97,2,31,1,97,2,2,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-13,10d9be,2,4,2,2,10,2,4,2,2020-04-06,2020-03-28,2020-04-12,1,1,43,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-13,11a9b3,2,4,9,2,9,15,58,1,2020-04-20,2020-04-20,9999-99-99,97,2,32,1,97,2,2,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-09-13,077c96,2,4,9,2,16,9,5,2,2020-04-30,2020-04-30,2020-05-15,1,1,66,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,1
4,2020-09-13,1719d0,2,4,30,1,30,30,193,1,2020-05-06,2020-05-01,2020-05-16,97,2,71,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97


Procedemos a cargar el archivo con los descriptores:

In [6]:
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

In [7]:
diccionario

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


Finalmente, cargamos el archivo de catálogos.

In [8]:
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

In [9]:
catalogues

{'MUNICIPIOS':       clave_municipio        municipio  clave_entidad
 0                   1   AGUASCALIENTES              1
 1                   2         ASIENTOS              1
 2                   3         CALVILLO              1
 3                   4            COSÍO              1
 4                   5      JESÚS MARÍA              1
 ...               ...              ...            ...
 2496              999  NO ESPECIFICADO             32
 2497              999  NO ESPECIFICADO             36
 2498              997        NO APLICA             97
 2499              998        SE IGNORA             98
 2500              999  NO ESPECIFICADO             99
 
 [2501 rows x 3 columns], 'NACIONALIDAD':    clave      descripción
 0      1         MEXICANA
 1      2       EXTRANJERA
 2     99  NO ESPECIFICADO, 'ORIGEN':    clave      descripción
 0      1            USMER
 1      2   FUERA DE USMER
 2     99  NO ESPECIFICADO, 'RESULTADO':   unnamed:_0              unnamed:_1
 0    

In [10]:
catalogues['SEXO']

,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


Perfilamiento


In [11]:
# dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1516588
Número de columnas: 35


In [12]:
# columnas (variables)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [13]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [14]:
# calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

In [15]:
# tipos de datos
data.dtypes

fecha_actualizacion    object
id_registro            object
origen                  int64
sector                  int64
entidad_um              int64
sexo                    int64
entidad_nac             int64
entidad_res             int64
municipio_res           int64
tipo_paciente           int64
fecha_ingreso          object
fecha_sintomas         object
fecha_def              object
intubado                int64
neumonia                int64
edad                    int64
nacionalidad            int64
embarazo                int64
habla_lengua_indig      int64
diabetes                int64
epoc                    int64
asma                    int64
inmusupr                int64
hipertension            int64
otra_com                int64
cardiovascular          int64
obesidad                int64
renal_cronica           int64
tabaquismo              int64
otro_caso               int64
resultado               int64
migrante                int64
pais_nacionalidad      object
pais_orige

Limpieza

In [16]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [17]:
data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-09-13,1e6e4c,2,4,25,2,25,25,11,1,2020-04-20,2020-04-17,NaT,97,2,31,1,97,2,2,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-13,10d9be,2,4,2,2,10,2,4,2,2020-04-06,2020-03-28,2020-04-12,1,1,43,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-13,11a9b3,2,4,9,2,9,15,58,1,2020-04-20,2020-04-20,NaT,97,2,32,1,97,2,2,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-09-13,077c96,2,4,9,2,16,9,5,2,2020-04-30,2020-04-30,2020-05-15,1,1,66,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,1
4,2020-09-13,1719d0,2,4,30,1,30,30,193,1,2020-05-06,2020-05-01,2020-05-16,97,2,71,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97


In [18]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [19]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [20]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

Corregimos el nombre de las columnas:

In [27]:
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

In [28]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

Removemos la (primera) fila que no nos es útil.

In [30]:
catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

KeyError: ignored

Convertimos el tipo de dato.

In [31]:
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

In [32]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [33]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [34]:
data.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-13,1e6e4c,2020-04-20,2020-04-17,NaT,31,MÃ©xico,99,SINALOA,GUASAVE,SINALOA,SINALOA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
1,2020-09-13,10d9be,2020-04-06,2020-03-28,2020-04-12,43,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,DURANGO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020
2,2020-09-13,11a9b3,2020-04-20,2020-04-20,NaT,32,MÃ©xico,99,MÉXICO,NEZAHUALCÓYOTL,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
3,2020-09-13,077c96,2020-04-30,2020-04-30,2020-05-15,66,MÃ©xico,99,CIUDAD DE MÉXICO,GUSTAVO A. MADERO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,4,2020
4,2020-09-13,1719d0,2020-05-06,2020-05-01,2020-05-16,71,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020


Filtros

In [35]:
# usando query
data.query('nacionalidad == "MEXICANA"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-13,1e6e4c,2020-04-20,2020-04-17,NaT,31,MÃ©xico,99,SINALOA,GUASAVE,SINALOA,SINALOA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
1,2020-09-13,10d9be,2020-04-06,2020-03-28,2020-04-12,43,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,DURANGO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


In [36]:
# haciendo referencia a la columna como índice
data[data['nacionalidad'] == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-13,1e6e4c,2020-04-20,2020-04-17,NaT,31,MÃ©xico,99,SINALOA,GUASAVE,SINALOA,SINALOA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
1,2020-09-13,10d9be,2020-04-06,2020-03-28,2020-04-12,43,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,DURANGO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


In [37]:
# seleccionando directamente la columna que queremos filtrar
data[data.nacionalidad == 'MEXICANA'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-13,1e6e4c,2020-04-20,2020-04-17,NaT,31,MÃ©xico,99,SINALOA,GUASAVE,SINALOA,SINALOA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,4,2020
1,2020-09-13,10d9be,2020-04-06,2020-03-28,2020-04-12,43,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,DURANGO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


Filtrado múltiple

In [38]:
# usando query podemos poner en el string del query todas las condiciones que queramos
data.query('nacionalidad == "MEXICANA" & entidad_res == "CIUDAD DE MÉXICO"').head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
3,2020-09-13,077c96,2020-04-30,2020-04-30,2020-05-15,66,MÃ©xico,99,CIUDAD DE MÉXICO,GUSTAVO A. MADERO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,4,2020
7,2020-09-13,100180,2020-05-06,2020-05-02,NaT,30,MÃ©xico,99,CIUDAD DE MÉXICO,CUAUHTÉMOC,CIUDAD DE MÉXICO,DURANGO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


In [39]:
# usando el filtrado tradicional debemos encerrar en paréntesis cada condición que queremos que cumpla nuestro filtro
data[(data['nacionalidad'] == "MEXICANA") & (data.entidad_res == "CIUDAD DE MÉXICO")].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
3,2020-09-13,077c96,2020-04-30,2020-04-30,2020-05-15,66,MÃ©xico,99,CIUDAD DE MÉXICO,GUSTAVO A. MADERO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,4,2020
7,2020-09-13,100180,2020-05-06,2020-05-02,NaT,30,MÃ©xico,99,CIUDAD DE MÉXICO,CUAUHTÉMOC,CIUDAD DE MÉXICO,DURANGO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020


Resúmenes

In [40]:
# una columna, una métrica
data.groupby('sexo').edad.mean()

sexo
HOMBRE    42.664726
MUJER     41.638688
Name: edad, dtype: float64

In [41]:
# una columna, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max']})

edad                
        min       mean  max
sexo                       
HOMBRE    0  42.664726  120
MUJER     0  41.638688  120

In [42]:
# varias columnas, una métrica
data.groupby('sexo').agg({'edad': 'mean', 'mes_ingreso': 'mean', 'mes_def': 'mean'})

,edad,mes_ingreso,mes_def
sexo,,,
HOMBRE,42.664726,6.739513,6.503351
MUJER,41.638688,6.816610,6.578603


In [43]:
# varias columnas, varias métricas
data.groupby('sexo').agg({'edad': ['min','mean','max'], 'mes_ingreso': ['min','mean','max'], 'mes_def': ['min','mean','max']})

edad                 mes_ingreso               mes_def               
        min       mean  max         min      mean max     min      mean  max
sexo                                                                        
HOMBRE    0  42.664726  120           1  6.739513   9     1.0  6.503351  9.0
MUJER     0  41.638688  120           1  6.816610   9     1.0  6.578603  9.0

Descripción por tipo.

In [44]:
# Filtrar por tipos generales (categóricas y numéricas)
numerical = data.select_dtypes(include='number')
categorical = data.select_dtypes(include=['object', 'bool', 'category', 'datetime64[ns]'])

In [45]:
numerical.head()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
0,31,NaN,NaN,4,2020
1,43,4.0,2020.0,4,2020
2,32,NaN,NaN,4,2020
3,66,5.0,2020.0,4,2020
4,71,5.0,2020.0,5,2020


In [46]:
categorical.head()

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion
0,2020-09-13,1e6e4c,2020-04-20,2020-04-17,NaT,MÃ©xico,99,SINALOA,GUASAVE,SINALOA,SINALOA,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
1,2020-09-13,10d9be,2020-04-06,2020-03-28,2020-04-12,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,DURANGO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True
2,2020-09-13,11a9b3,2020-04-20,2020-04-20,NaT,MÃ©xico,99,MÉXICO,NEZAHUALCÓYOTL,CIUDAD DE MÉXICO,CIUDAD DE MÉXICO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False
3,2020-09-13,077c96,2020-04-30,2020-04-30,2020-05-15,MÃ©xico,99,CIUDAD DE MÉXICO,GUSTAVO A. MADERO,CIUDAD DE MÉXICO,MICHOACÁN DE OCAMPO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True
4,2020-09-13,1719d0,2020-05-06,2020-05-01,2020-05-16,MÃ©xico,99,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,IMSS,MUJER,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,True


In [47]:
# para conocer todos los tipo de datos
?data.select_dtypes

Categóricas

In [48]:
# Cardinalidad - Valores únicos por columna
for col in categorical.columns:
  print('Valores únicos en "{}": {}'.format(col,categorical[col].nunique()))

Valores únicos en "fecha_actualizacion": 1
Valores únicos en "id_registro": 1516588
Valores únicos en "fecha_ingreso": 257
Valores únicos en "fecha_sintomas": 257
Valores únicos en "fecha_def": 202
Valores únicos en "pais_nacionalidad": 119
Valores únicos en "pais_origen": 77
Valores únicos en "entidad_res": 32
Valores únicos en "municipio_res": 2164
Valores únicos en "entidad_um": 32
Valores únicos en "entidad_nac": 33
Valores únicos en "intubado": 4
Valores únicos en "neumonia": 3
Valores únicos en "embarazo": 4
Valores únicos en "habla_lengua_indig": 3
Valores únicos en "diabetes": 3
Valores únicos en "epoc": 3
Valores únicos en "asma": 3
Valores únicos en "inmusupr": 3
Valores únicos en "hipertension": 3
Valores únicos en "otra_com": 3
Valores únicos en "cardiovascular": 3
Valores únicos en "obesidad": 3
Valores únicos en "renal_cronica": 3
Valores únicos en "tabaquismo": 3
Valores únicos en "otro_caso": 3
Valores únicos en "migrante": 3
Valores únicos en "uci": 4
Valores únicos en

In [49]:
# Unicidad - proporción de valores únicos por columna
total = data.shape[0]
for col in categorical.columns:
  print('Porcentaje de valores únicos en "{}": {}%'.format(col,round(categorical[col].nunique()/total*100,2)))

Porcentaje de valores únicos en "fecha_actualizacion": 0.0%
Porcentaje de valores únicos en "id_registro": 100.0%
Porcentaje de valores únicos en "fecha_ingreso": 0.02%
Porcentaje de valores únicos en "fecha_sintomas": 0.02%
Porcentaje de valores únicos en "fecha_def": 0.01%
Porcentaje de valores únicos en "pais_nacionalidad": 0.01%
Porcentaje de valores únicos en "pais_origen": 0.01%
Porcentaje de valores únicos en "entidad_res": 0.0%
Porcentaje de valores únicos en "municipio_res": 0.14%
Porcentaje de valores únicos en "entidad_um": 0.0%
Porcentaje de valores únicos en "entidad_nac": 0.0%
Porcentaje de valores únicos en "intubado": 0.0%
Porcentaje de valores únicos en "neumonia": 0.0%
Porcentaje de valores únicos en "embarazo": 0.0%
Porcentaje de valores únicos en "habla_lengua_indig": 0.0%
Porcentaje de valores únicos en "diabetes": 0.0%
Porcentaje de valores únicos en "epoc": 0.0%
Porcentaje de valores únicos en "asma": 0.0%
Porcentaje de valores únicos en "inmusupr": 0.0%
Porcenta

In [50]:
# Moda - valor que más se repite en cada columna excepto los indicadores únicos
for col in categorical.drop(columns='id_registro').columns:
  print('Valor más popular en "{}": {}'.format(col, statistics.mode(categorical[col])))

Valor más popular en "fecha_actualizacion": 2020-09-13 00:00:00
Valor más popular en "fecha_ingreso": 2020-07-20 00:00:00
Valor más popular en "fecha_sintomas": 2020-07-20 00:00:00
Valor más popular en "fecha_def": NaT
Valor más popular en "pais_nacionalidad": MÃ©xico
Valor más popular en "pais_origen": 99
Valor más popular en "entidad_res": CIUDAD DE MÉXICO
Valor más popular en "municipio_res": IZTAPALAPA
Valor más popular en "entidad_um": CIUDAD DE MÉXICO
Valor más popular en "entidad_nac": CIUDAD DE MÉXICO
Valor más popular en "intubado": NO APLICA
Valor más popular en "neumonia": NO 
Valor más popular en "embarazo": NO 
Valor más popular en "habla_lengua_indig": NO 
Valor más popular en "diabetes": NO 
Valor más popular en "epoc": NO 
Valor más popular en "asma": NO 
Valor más popular en "inmusupr": NO 
Valor más popular en "hipertension": NO 
Valor más popular en "otra_com": NO 
Valor más popular en "cardiovascular": NO 
Valor más popular en "obesidad": NO 
Valor más popular en "r

In [51]:
numerical.describe()

,edad,mes_def,año_def,mes_ingreso,año_ingreso
count,1.516588e+06,90451.000000,90451.0,1.516588e+06,1516588.0
mean,4.214249e+01,6.530818,2020.0,6.778754e+00,2020.0
std,1.681217e+01,1.310083,0.0,1.400038e+00,0.0
min,0.000000e+00,1.000000,2020.0,1.000000e+00,2020.0
25%,3.000000e+01,6.000000,2020.0,6.000000e+00,2020.0
50%,4.100000e+01,7.000000,2020.0,7.000000e+00,2020.0
75%,5.300000e+01,8.000000,2020.0,8.000000e+00,2020.0
max,1.200000e+02,9.000000,2020.0,9.000000e+00,2020.0


Análisis Exploratorio

In [52]:
# ¿Cuántas pruebas por origen se realizan?
data.groupby('origen').id_registro.count()

origen
FUERA DE USMER    1015089
USMER              501499
Name: id_registro, dtype: int64

In [53]:
data.origen.value_counts()

FUERA DE USMER    1015089
USMER              501499
Name: origen, dtype: int64

In [54]:
# ¿Cuántas pruebas por sector se realizan?
data.sector.value_counts()

SSA                946933
IMSS               387379
ISSSTE              57584
PRIVADA             55814
ESTATAL             27221
PEMEX               12946
IMSS-BIENESTAR      10162
SEMAR                7808
SEDENA               7412
MUNICIPAL            1203
UNIVERSITARIO        1047
DIF                   849
CRUZ ROJA             222
NO ESPECIFICADO         8
Name: sector, dtype: int64

In [55]:
# ¿Cuántos pacientes por sexo tenemos?
data.sexo.value_counts()

MUJER     771914
HOMBRE    744674
Name: sexo, dtype: int64

In [56]:
# ¿Cuál es la proporción de los pacientes por sexo?
data.sexo.value_counts(normalize=True)

MUJER     0.508981
HOMBRE    0.491019
Name: sexo, dtype: float64

In [57]:
# ¿Cuál es la proporción de resultados de las pruebas?
data.resultado.value_counts(normalize=True)

No positivo SARS-CoV-2    0.504644
Positivo SARS-CoV-2       0.440714
Resultado pendiente       0.054642
Name: resultado, dtype: float64

In [58]:
# ¿Cuántas pruebas positivas tenemos en el set?
data[data.resultado == 'Positivo SARS-CoV-2'].id_registro.count()

668381

In [59]:
# ¿Cuántos casos positivos tenemos por mes?
data[data.resultado == 'Positivo SARS-CoV-2'].mes_ingreso.value_counts().sort_index()

1         2
2         8
3      2530
4     26574
5     86561
6    153132
7    199721
8    156160
9     43693
Name: mes_ingreso, dtype: int64

In [60]:
# ¿Cuál es el resultado de las pruebas por sexo?
data.groupby(['sexo','resultado'])['id_registro'].count()

sexo    resultado             
HOMBRE  No positivo SARS-CoV-2    355413
        Positivo SARS-CoV-2       348621
        Resultado pendiente        40640
MUJER   No positivo SARS-CoV-2    409924
        Positivo SARS-CoV-2       319760
        Resultado pendiente        42230
Name: id_registro, dtype: int64

In [61]:
data.groupby('sexo').resultado.value_counts()

sexo    resultado             
HOMBRE  No positivo SARS-CoV-2    355413
        Positivo SARS-CoV-2       348621
        Resultado pendiente        40640
MUJER   No positivo SARS-CoV-2    409924
        Positivo SARS-CoV-2       319760
        Resultado pendiente        42230
Name: resultado, dtype: int64

In [62]:
# ¿Qué proporción de resultados de pruebas tenemos por sexo?
data.groupby('sexo').resultado.value_counts(normalize=True)

sexo    resultado             
HOMBRE  No positivo SARS-CoV-2    0.477273
        Positivo SARS-CoV-2       0.468153
        Resultado pendiente       0.054574
MUJER   No positivo SARS-CoV-2    0.531049
        Positivo SARS-CoV-2       0.414243
        Resultado pendiente       0.054708
Name: resultado, dtype: float64

In [63]:
# ¿Cuál es la edad de los pacientes por sexo?
data.groupby('sexo').edad.describe()

,count,mean,std,min,25%,50%,75%,max
sexo,,,,,,,,
HOMBRE,744674.0,42.664726,17.130820,0.0,30.0,41.0,54.0,120.0
MUJER,771914.0,41.638688,16.483266,0.0,30.0,40.0,52.0,120.0


In [64]:
def q25(x):
  return x.quantile(0.25)

def q75(x):
  return x.quantile(0.75)

data.groupby('sexo').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                    
        min q25 median q75  max
sexo                           
HOMBRE    0  30     41  54  120
MUJER     0  30     40  52  120

In [65]:
# ¿Cuál es la edad de los pacientes dependiendo de si son intubados o no?
data.groupby('intubado').edad.describe()

,count,mean,std,min,25%,50%,75%,max
intubado,,,,,,,,
NO,222376.0,53.310371,19.689511,0.0,42.0,55.0,67.00,118.0
NO APLICA,1254306.0,39.686841,15.085025,0.0,29.0,38.0,49.00,120.0
NO ESPECIFICADO,192.0,54.427083,16.901776,0.0,45.0,57.0,65.25,92.0
SI,39714.0,57.107267,17.917199,0.0,48.0,59.0,69.00,108.0


In [66]:
data.groupby('intubado').agg({'edad': ['min', q25, 'median', q75, 'max']})

edad                       
                 min q25 median    q75  max
intubado                                   
NO                 0  42     55  67.00  118
NO APLICA          0  29     38  49.00  120
NO ESPECIFICADO    0  45     57  65.25   92
SI                 0  48     59  69.00  108

In [67]:
# ¿Cuál es la edad máxima y cuál es la mínima de los pacientes fallecidos?
edad_def_max = data[data.defuncion].edad.max()
data[data.defuncion & (data.edad == edad_def_max)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
563541,2020-09-13,1b7b20,2020-05-21,2020-05-20,2020-05-26,109,MÃ©xico,99,MICHOACÁN DE OCAMPO,APATZINGÁN,MICHOACÁN DE OCAMPO,MICHOACÁN DE OCAMPO,NO,SI,NO APLICA,NO,NO,NO,NO,SI,NO,NO,NO,NO,SI,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ISSSTE,HOMBRE,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,5.0,2020.0,5,2020


In [68]:
edad_def_min = data[data.defuncion].edad.min()
data[data.defuncion & (data.edad == edad_def_min)]

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
1695,2020-09-13,083c34,2020-05-25,2020-05-25,2020-06-01,0,MÃ©xico,99,MÉXICO,CHIMALHUACÁN,MÉXICO,MÉXICO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,6.0,2020.0,5,2020
12925,2020-09-13,02f8e2,2020-05-08,2020-05-01,2020-05-10,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
37588,2020-09-13,053398,2020-06-11,2020-06-11,2020-06-17,0,MÃ©xico,99,AGUASCALIENTES,AGUASCALIENTES,AGUASCALIENTES,AGUASCALIENTES,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,6.0,2020.0,6,2020
47051,2020-09-13,1b82ee,2020-05-13,2020-05-13,2020-05-22,0,MÃ©xico,99,MÉXICO,ECATEPEC DE MORELOS,CIUDAD DE MÉXICO,MÉXICO,SI,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,FUERA DE USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,5.0,2020.0,5,2020
51755,2020-09-13,18feee,2020-06-23,2020-06-23,2020-07-06,0,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,BAJA CALIFORNIA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,IMSS,MUJER,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,7.0,2020.0,6,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442665,2020-09-13,09bf2f,2020-07-28,2020-07-28,2020-08-05,0,MÃ©xico,99,QUINTANA ROO,SOLIDARIDAD,QUINTANA ROO,QUINTANA ROO,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,SI,USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,No positivo SARS-CoV-2,True,8.0,2020.0,7,2020
1472228,2020-09-13,1e94e4,2020-08-14,2020-08-06,2020-08-20,0,MÃ©xico,99,SAN LUIS POTOSÍ,SAN LUIS POTOSÍ,SAN LUIS POTOSÍ,SAN LUIS POTOSÍ,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,NO ESPECIFICADO,SI,FUERA DE USMER,PRIVADA,HOMBRE,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,8.0,2020.0,8,2020
1480736,2020-09-13,00f39c,2020-07-06,2020-07-03,2020-07-08,0,MÃ©xico,99,JALISCO,ZAPOTILTIC,JALISCO,JALISCO,NO,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,SI,USMER,SSA,HOMBRE,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,7.0,2020.0,7,2020
1503360,2020-09-13,07cf0b,2020-08-26,2020-08-25,2020-08-27,0,MÃ©xico,99,CHIHUAHUA,JUÁREZ,CHIHUAHUA,COAHUILA DE ZARAGOZA,NO,SI,NO,NO,NO,NO,NO,NO,NO,SE IGNORA,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,USMER,SSA,MUJER,HOSPITALIZADO,MEXICANA,Resultado pendiente,True,8.0,2020.0,8,2020


In [69]:
# ¿Cuál es la edad de los pacientes que fallecen?
data.groupby('defuncion').edad.describe()

,count,mean,std,min,25%,50%,75%,max
defuncion,,,,,,,,
False,1426137.0,40.888726,16.110050,0.0,29.0,39.0,51.0,120.0
True,90451.0,61.910570,15.216661,0.0,53.0,63.0,73.0,109.0


In [70]:
# ¿Cuántas defunciones tenemos por mes?
data[data.defuncion].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      232
4.0     5294
5.0    16096
6.0    21469
7.0    23861
8.0    18833
9.0     4648
Name: mes_def, dtype: int64

In [71]:
# ¿Cuál es la proporción de diábeticos para las defunciones?
data.groupby('diabetes').defuncion.value_counts(normalize=True)

diabetes   defuncion
NO         False        0.958202
           True         0.041798
SE IGNORA  False        0.872792
           True         0.127208
SI         False        0.813552
           True         0.186448
Name: defuncion, dtype: float64

In [72]:
# ¿Cuál es la proporción de asmáticos para las defunciones?
data.groupby('asma').defuncion.value_counts(normalize=True)

asma       defuncion
NO         False        0.940037
           True         0.059963
SE IGNORA  False        0.866152
           True         0.133848
SI         False        0.958092
           True         0.041908
Name: defuncion, dtype: float64

**1. ¿Tienen los pacientes con hipertensión un riesgo más alto de defunción?**

In [73]:
data.groupby('hipertension').defuncion.value_counts(normalize=True)

hipertension  defuncion
NO            False        0.960811
              True         0.039189
SE IGNORA     False        0.872118
              True         0.127882
SI            False        0.834710
              True         0.165290
Name: defuncion, dtype: float64

**2. ¿Cuántos casos confirmados se tienen por Estado?**

In [77]:
data.query('resultado == "Positivo SARS-CoV-2"').groupby("resultado").entidad_res.value_counts().reset_index(name="positivos_covid").drop(columns="resultado")

,entidad_res,positivos_covid
0,CIUDAD DE MÉXICO,111443
1,MÉXICO,74067
2,GUANAJUATO,36646
3,NUEVO LEÓN,33926
4,VERACRUZ DE IGNACIO DE LA LLAVE,30867
5,TABASCO,30045
6,PUEBLA,29129
7,TAMAULIPAS,26861
8,COAHUILA DE ZARAGOZA,24043
9,JALISCO,23221


**3. ¿Cuántas defunciones se tienen por Estado?**

In [79]:
data.query('defuncion == True').groupby("defuncion").entidad_res.value_counts().reset_index(name="defunciones").drop(columns="defuncion")

,entidad_res,defunciones
0,MÉXICO,13682
1,CIUDAD DE MÉXICO,12038
2,VERACRUZ DE IGNACIO DE LA LLAVE,4861
3,PUEBLA,4636
4,BAJA CALIFORNIA,4085
5,JALISCO,3918
6,SINALOA,3759
7,NUEVO LEÓN,3558
8,SONORA,3284
9,GUANAJUATO,3222


**4. ¿Cuántos fallecimientos han ocurrido en el Estado con mayor número de casos confirmados?**

In [83]:
max(data.query('defuncion == True').groupby("defuncion").entidad_res.value_counts())

13682

**5. ¿Cuántos fallecimientos han ocurrido en los pacientes Ambulatorios?**

In [91]:
data.query('defuncion == True and tipo_paciente == "AMBULATORIO"').tipo_paciente.value_counts().iloc[0]

9633